In [1]:
# Template Notebook for Engagement Classification Model

# ToDo/Models to run here: https://docs.google.com/document/d/18EJpFesMEl9Q7C1AZzDeq6uy7c8tDMvEJ_j58-MmiBI/edit?usp=sharing
# Use p8_data_processed.csv here: https://drive.google.com/drive/folders/19aJUAlkTMz7PcZE1q4hogFkjVtwYGcMT
# Upload code to help-seeking github: https://github.com/interaction-lab/help_seeking
# Record model results here: https://docs.google.com/spreadsheets/d/16ye54fSSEuAuDL_j56UIeDB-rIIrxq_kPbtyPRQOrVI/edit?usp=sharing

In [2]:
# Imports
import pandas as pd
import numpy as np
from pandas import *
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [3]:
# Load Data
file8 = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4/p8_data_FS4.csv'
file3 = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4/p3_data_FS4.csv'
file7 = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4/p7_data_FS4.csv'
file5 = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4/p5_data_FS4.csv'
file9 = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4/p9_data_FS4.csv'
file10 = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Help-Seeking/Data/FS4/p10_data_FS4.csv'

data8 = pd.read_csv(file8)
data3 = pd.read_csv(file3)
data7 = pd.read_csv(file7)
data5 = pd.read_csv(file5)
data9 = pd.read_csv(file9)
data10 = pd.read_csv(file10)

In [46]:
data7.head()

,of_confidence,of_success,of_gaze_0_x,of_gaze_0_y,of_gaze_0_z,of_gaze_1_x,of_gaze_1_y,of_gaze_1_z,of_gaze_angle_x,of_gaze_angle_y,...,ts_attempt,skill_NC,skill_OS,skill_EM,diff_1,diff_2,diff_3,diff_4,diff_5,no_game
0,1.0,1.0,0.802148,0.562868,0.109746,0.394162,0.568244,0.018943,0.606309,0.508006,...,0.0,0,1,0,0,1,0,0,0,0
1,1.0,1.0,0.805980,0.553090,0.111663,0.395043,0.564921,0.018227,0.607571,0.505478,...,0.0,0,1,0,0,1,0,0,0,0
2,1.0,1.0,0.802828,0.542876,0.108154,0.413679,0.556325,0.012879,0.610515,0.501938,...,0.0,0,1,0,0,1,0,0,0,0
3,1.0,1.0,0.806615,0.533710,0.110562,0.390660,0.568724,0.019901,0.606730,0.502613,...,0.0,0,1,0,0,1,0,0,0,0
4,1.0,1.0,0.805455,0.529496,0.109395,0.397752,0.560606,0.016963,0.607781,0.500253,...,0.0,0,1,0,0,1,0,0,0,0


Note: keep timestamp and session_num until right before running the model in all feature sets

### Step 1: Train Test Split 

Train on Earlier Sessions, Test on Later Sessions 70-30

In [64]:
# Feature Set 4 is used for each participant in set
FS = data10

In [65]:
# sort values chronologically
FS = FS.sort_values(['session_num', 'timestamp'])

In [66]:
vals = []
for val in FS['session_num']:
    if val not in vals:
        vals.append(val)
        print(val)

2.1
2.2
3.1
3.2
4.2
4.3
4.4
5.1
5.2
7.1
7.2
8.0
9.0
10.1
10.3
11.0
12.1
12.2
13.0
14.2
15.1
15.2
16.0
17.1
17.2
18.2
18.3
19.0
20.3
21.0
22.2


In [40]:
# session thresholds for 70 30 train-test split
session_threshold_p8 = 8
session_threshold_p3 = 29
session_threshold_p7 = 17
session_threshold_p5 = 13
session_threshold_p9 = 9
session_threshold_p10 = 15.2

In [9]:
def session_split(sess_threshold):
    train = FS[(FS['session_num'] <= sess_threshold)] 
    test = FS[(FS['session_num'] > sess_threshold)]

    # drop columns needed for split
    train2 = train.drop(columns=['timestamp'])
    test2 = test.drop(columns=['timestamp'])

    X_train2 = train2.drop(columns=['engagement'])
    y_train2 = train2['engagement']

    X_test2 = test2.drop(columns=['engagement'])
    y_test2 = test2['engagement']
    
    return X_train2, y_train2, X_test2, y_test2

In [67]:
X_train2, y_train2, X_test2, y_test2 = session_split(session_threshold_p10)

### Step 2: Implement the HMM

In [11]:
from hmmlearn import hmm
from hmmlearn.hmm import GaussianHMM
# 3 questions from HMM:
# (1) given model, what is likelihood of sequence S happening?
# (2) given model and sequence S, what is optimal hidden state sequence?
# (3) given sequence S and # of hidden states, what is optimal model to maximize probability of S?

### P8 FS4

In [12]:
# create lengths array of session in train data
all_lengths_8 = X_train2['session_num'].value_counts(sort=False).tolist()
X_train2 = X_train2.drop(columns=['session_num'])

In [13]:
# model takes array of dimension (n_samples, n_features)
model_p8 = GaussianHMM(n_components=2, n_iter=1000, algorithm = 'viterbi', verbose = True).fit(X_train2, all_lengths_8)
print(model_p8.monitor_.history)

         1     1010234.3618             +nan
         2     2417962.5265    +1407728.1647
         3     2597384.2574     +179421.7309
         4     2673304.2101      +75919.9526
         5     2760566.2938      +87262.0837
         6     2782146.9887      +21580.6949
         7     2824261.8818      +42114.8931
         8     2916239.6865      +91977.8047
         9     2954788.3400      +38548.6535
        10     2965564.6379      +10776.2978
        11     2965982.3034        +417.6655
        12     2966010.5360         +28.2326
        13     2966012.3163          +1.7804
        14     2966012.4634          +0.1471


deque([2966012.476987693, 2966012.4782911385], maxlen=2)


        15     2966012.4770          +0.0135
        16     2966012.4783          +0.0013


In [19]:
all_lengths_8_test = X_test2['session_num'].value_counts(sort=False).tolist()
X_test2 = X_test2.drop(columns=['session_num'])

In [26]:
#evaluate FS4 P8 HMM
FS4_pred_p8 = model_p8.predict(X_test2, all_lengths_8_test)
scores2 = model_p8.predict_proba(X_test2.values, all_lengths_8_test)
scores2 = scores2[: ,1]
print("Accuracy for FS4 P8 HMM:",accuracy_score(y_test2, FS4_pred_p8))
print("AUC:",roc_auc_score(y_test2, scores2)) 
print(metrics.confusion_matrix(y_test2, FS4_pred_p8))
print(metrics.classification_report(y_test2, FS4_pred_p8))

Accuracy for FS4 P8 HMM: 0.577750923001
AUC: 0.692824033797
[[41603 22974]
 [21401 19114]]
             precision    recall  f1-score   support

        0.0       0.66      0.64      0.65     64577
        1.0       0.45      0.47      0.46     40515

avg / total       0.58      0.58      0.58    105092



### P3 FS4

In [31]:
X_train2.shape

(86393, 30)

In [33]:
# create lengths array of session in train data
all_lengths_3 = X_train2['session_num'].value_counts(sort=False).tolist()
X_train2 = X_train2.drop(columns=['session_num'])

In [34]:
# model takes array of dimension (n_samples, n_features)
model_p3 = GaussianHMM(n_components=2, n_iter=1000, algorithm = 'viterbi', verbose = True).fit(X_train2, all_lengths_3)
print(model_p3.monitor_.history)

         1      334724.1304             +nan
         2      642242.4302     +307518.2998
         3      716427.2887      +74184.8585
         4      863640.7308     +147213.4420
         5      990225.0465     +126584.3157


deque([1211899.5449388099, 1211899.54493902], maxlen=2)


         6     1211899.5449     +221674.4985
         7     1211899.5449          +0.0000


In [35]:
all_lengths_3_test = X_test2['session_num'].value_counts(sort=False).tolist()
X_test2 = X_test2.drop(columns=['session_num'])

In [36]:
#evaluate FS4 P3 HMM
FS4_pred_p3 = model_p3.predict(X_test2, all_lengths_3_test)
scores2 = model_p3.predict_proba(X_test2.values, all_lengths_3_test)
scores2 = scores2[: ,1]
print("Accuracy for FS4 P3 HMM:",accuracy_score(y_test2, FS4_pred_p3))
print("AUC:",roc_auc_score(y_test2, scores2)) 
print(metrics.confusion_matrix(y_test2, FS4_pred_p3))
print(metrics.classification_report(y_test2, FS4_pred_p3))

Accuracy for FS4 P3 HMM: 0.71677864875
AUC: 0.7386119225
[[ 7173  2118]
 [13057 31232]]
             precision    recall  f1-score   support

        0.0       0.35      0.77      0.49      9291
        1.0       0.94      0.71      0.80     44289

avg / total       0.84      0.72      0.75     53580



### P7 FS4

In [42]:
# create lengths array of session in train data
all_lengths_7 = X_train2['session_num'].value_counts(sort=False).tolist()
X_train2 = X_train2.drop(columns=['session_num'])

In [43]:
# model takes array of dimension (n_samples, n_features)
model_p7 = GaussianHMM(n_components=2, n_iter=1000, algorithm = 'viterbi', verbose = True).fit(X_train2, all_lengths_7)
print(model_p7.monitor_.history)

         1     2109038.8848             +nan
         2     4765060.7865    +2656021.9017
         3     7568081.6547    +2803020.8682


deque([7568081.6546588773, 7568081.6546559846], maxlen=2)


         4     7568081.6547          -0.0000


In [44]:
all_lengths_7_test = X_test2['session_num'].value_counts(sort=False).tolist()
X_test2 = X_test2.drop(columns=['session_num'])

In [45]:
#evaluate FS4 P7 HMM
FS4_pred_p7 = model_p7.predict(X_test2, all_lengths_7_test)
scores2 = model_p7.predict_proba(X_test2.values, all_lengths_7_test)
scores2 = scores2[: ,1]
print("Accuracy for FS4 P7 HMM:",accuracy_score(y_test2, FS4_pred_p7))
print("AUC:",roc_auc_score(y_test2, scores2)) 
print(metrics.confusion_matrix(y_test2, FS4_pred_p7))
print(metrics.classification_report(y_test2, FS4_pred_p7))

Accuracy for FS4 P7 HMM: 0.166443598181
AUC: 0.188475758366
[[ 1217 75036]
 [37690 21292]]
             precision    recall  f1-score   support

        0.0       0.03      0.02      0.02     76253
        1.0       0.22      0.36      0.27     58982

avg / total       0.11      0.17      0.13    135235



### P5 FS4 HMM

In [51]:
# create lengths array of session in train data
all_lengths_5 = X_train2['session_num'].value_counts(sort=False).tolist()
X_train2 = X_train2.drop(columns=['session_num'])

In [52]:
# model takes array of dimension (n_samples, n_features)
model_p5 = GaussianHMM(n_components=2, n_iter=1000, algorithm = 'viterbi', verbose = True).fit(X_train2, all_lengths_5)
print(model_p5.monitor_.history)

         1     2090301.3049             +nan
         2     6467552.6495    +4377251.3446
         3    10259784.6480    +3792231.9985
         4    11318995.9562    +1059211.3082


deque([11318995.956174908, 11318995.956170959], maxlen=2)


         5    11318995.9562          -0.0000


In [53]:
all_lengths_5_test = X_test2['session_num'].value_counts(sort=False).tolist()
X_test2 = X_test2.drop(columns=['session_num'])

In [54]:
#evaluate FS4 P5 HMM
FS4_pred_p5 = model_p5.predict(X_test2, all_lengths_5_test)
scores2 = model_p5.predict_proba(X_test2.values, all_lengths_5_test)
scores2 = scores2[: ,1]
print("Accuracy for FS4 P5 HMM:",accuracy_score(y_test2, FS4_pred_p5))
print("AUC:",roc_auc_score(y_test2, scores2)) 
print(metrics.confusion_matrix(y_test2, FS4_pred_p5))
print(metrics.classification_report(y_test2, FS4_pred_p5))

Accuracy for FS4 P5 HMM: 0.299378691417
AUC: 0.304511285228
[[ 2839  6249]
 [30287 12773]]
             precision    recall  f1-score   support

          0       0.09      0.31      0.13      9088
          1       0.67      0.30      0.41     43060

avg / total       0.57      0.30      0.36     52148



### P9 FS4 HMM

In [59]:
# create lengths array of session in train data
all_lengths_9 = X_train2['session_num'].value_counts(sort=False).tolist()
X_train2 = X_train2.drop(columns=['session_num'])

In [60]:
# model takes array of dimension (n_samples, n_features)
model_p9 = GaussianHMM(n_components=2, n_iter=1000, algorithm = 'viterbi', verbose = True).fit(X_train2, all_lengths_9)
print(model_p9.monitor_.history)

         1     1326061.2378             +nan
         2     2427889.4972    +1101828.2594
         3     4666415.5990    +2238526.1018
         4     4964029.4827     +297613.8837
         5     6132451.9192    +1168422.4365
         6     6875929.1679     +743477.2487
         7     7378169.6439     +502240.4760
         8     7378451.7859        +282.1421


deque([7378451.7859358806, 7378451.7859390574], maxlen=2)


         9     7378451.7859          +0.0000


In [61]:
all_lengths_9_test = X_test2['session_num'].value_counts(sort=False).tolist()
X_test2 = X_test2.drop(columns=['session_num'])

In [63]:
#evaluate FS4 P9 HMM
FS4_pred_p9 = model_p5.predict(X_test2, all_lengths_9_test)
scores2 = model_p9.predict_proba(X_test2.values, all_lengths_9_test)
scores2 = scores2[: ,1]
print("Accuracy for FS4 P9 HMM:",accuracy_score(y_test2, FS4_pred_p9))
print("AUC:",roc_auc_score(y_test2, scores2)) 
print(metrics.confusion_matrix(y_test2, FS4_pred_p9))
print(metrics.classification_report(y_test2, FS4_pred_p9))

Accuracy for FS4 P9 HMM: 0.296315102674
AUC: 0.497699591787
[[ 8691 98549]
 [30333 45580]]
             precision    recall  f1-score   support

          0       0.22      0.08      0.12    107240
          1       0.32      0.60      0.41     75913

avg / total       0.26      0.30      0.24    183153



### P10 FS4 HMM

In [68]:
# create lengths array of session in train data
all_lengths_10 = X_train2['session_num'].value_counts(sort=False).tolist()
X_train2 = X_train2.drop(columns=['session_num'])

In [69]:
# model takes array of dimension (n_samples, n_features)
model_p10 = GaussianHMM(n_components=2, n_iter=1000, algorithm = 'viterbi', verbose = True).fit(X_train2, all_lengths_10)
print(model_p10.monitor_.history)

         1     3142183.2835             +nan
         2     5366809.5093    +2224626.2257
         3     6191427.6169     +824618.1076
         4     7005523.4161     +814095.7992
         5     8044629.0339    +1039105.6178
         6     9820489.8108    +1775860.7769
         7    15006479.4096    +5185989.5988
         8    25732219.7503   +10725740.3407


deque([25732219.750330932, 25732219.750343643], maxlen=2)


         9    25732219.7503          +0.0000


In [70]:
all_lengths_10_test = X_test2['session_num'].value_counts(sort=False).tolist()
X_test2 = X_test2.drop(columns=['session_num'])

In [72]:
#evaluate FS4 P10 HMM
FS4_pred_p10 = model_p10.predict(X_test2, all_lengths_10_test)
scores2 = model_p10.predict_proba(X_test2.values, all_lengths_10_test)
scores2 = scores2[: ,1]
print("Accuracy for FS4 P10 HMM:",accuracy_score(y_test2, FS4_pred_p10))
print("AUC:",roc_auc_score(y_test2, scores2)) 
print(metrics.confusion_matrix(y_test2, FS4_pred_p10))
print(metrics.classification_report(y_test2, FS4_pred_p10))

Accuracy for FS4 P10 HMM: 0.677394545347
AUC: 0.643238102732
[[38091 35070]
 [30366 99309]]
             precision    recall  f1-score   support

        0.0       0.56      0.52      0.54     73161
        1.0       0.74      0.77      0.75    129675

avg / total       0.67      0.68      0.67    202836

